In [158]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def get_products_inf(soup):
    data_products = []
    if (temp_soup.find('h1').text == 'Pescado fresco'):
        sections = soup.find_all('div', class_='category-section')
        for section in sections:
            type_product = section.find('h3').text
            products = section.find_all('div', attrs={'data-testid' : 'product-cell'})
            for product in products:
                img = product.find('img')['src']
                info = product.find('div', class_='product-cell__info')
                label = info.find('h4').text
                sub_label = ''.join([sub_label_inv.text for sub_label_inv in info.find_all('span')])
                prices = product.find('div', class_='product-price')
                try:
                    price_before_discount = prices.find('p', class_='product-price__previous-unit-price footnote1-r').text
                    price_before_discount = price_before_discount.split()[0]
                    
                except:
                    price_before_discount = None
                
                try:
                    price_with_discount = prices.find('p', class_='product-price__unit-price subhead1-b product-price__unit-price--discount').text
                    price_with_discount = price_with_discount.split()[0]
                except:
                    price_with_discount = None
                try:
                    quantity = prices.find('p', class_='product-price__extra-price subhead1-r').text
                except:
                    quantity = None
                try:
                    real_price = prices.find('p', class_='product-price__unit-price subhead1-b').text
                    real_price = real_price.split()[0]

                except:
                    real_price = None
                
                data_products.append([type_product, label, sub_label, price_with_discount, price_before_discount, real_price, quantity, img])
    else:
        sections = soup.find_all('section', attrs={'class' : 'section'})
        for section in sections:
            type_product = section.find('h2').text
            products = section.find_all('div', attrs={'data-testid' : 'product-cell'})
            for product in products:
                img = product.find('img')['src']
                info = product.find('div', class_='product-cell__info')
                label = info.find('h4').text
                sub_label = ''.join([sub_label_inv.text for sub_label_inv in info.find_all('span')])
                prices = product.find('div', class_='product-price')
                try:
                    price_before_discount = prices.find('p', class_='product-price__previous-unit-price footnote1-r').text
                    price_before_discount = price_before_discount.split()[0]
                    
                except:
                    price_before_discount = None
                
                try:
                    price_with_discount = prices.find('p', class_='product-price__unit-price subhead1-b product-price__unit-price--discount').text
                    price_with_discount = price_with_discount.split()[0]
                except:
                    price_with_discount = None
                try:
                    quantity = prices.find('p', class_='product-price__extra-price subhead1-r').text
                except:
                    quantity = None
                try:
                    real_price = prices.find('p', class_='product-price__unit-price subhead1-b').text
                    real_price = real_price.split()[0]

                except:
                    real_price = None
                
                data_products.append([type_product, label, sub_label, price_with_discount, price_before_discount, real_price, quantity, img])
    return data_products
            

df = pd.DataFrame(columns=['category', 'subcategory', 'type', 'label', 'sub_label', 'price_with_discount', 'price_before_discount', 'real_price', 'quantity', 'img'])
# Configuración de opciones para el navegador (opcional)
options = Options()
options.add_argument("--start-maximized")  # Abrir el navegador maximizado

# Crear una instancia del navegador usando el WebDriver Manager incorporado
driver = webdriver.Chrome(options=options)

# Abrir una página web
driver.get('https://tienda.mercadona.es/categories/112')
wait = WebDriverWait(driver, 10)  # Esperar un máximo de 10 segundos
campo_codigo_postal = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name='postalCode']")))

# Introducir el código postal
campo_codigo_postal.send_keys("28017")

# Esperar hasta que el botón de continuar esté presente y visible
boton_continuar = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='postal-code-checker-button']")))

# Hacer clic en el botón de continuar
boton_continuar.click()
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


categories = soup.find_all('li', class_='category-menu__item')
for i, category in enumerate(categories):

    category_name = category.find('span', class_='category-menu__header').find('label').text
    categories = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.category-menu__item")))

    button_category = categories[i]
    button_category.click()

    temp_html = driver.page_source
    temp_soup = BeautifulSoup(temp_html, 'html.parser')

    for j, subcategory in enumerate(temp_soup.find_all('li', class_='subhead1-r category-item')):
        subcategory_name = subcategory.find('button').text

        subcategories_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".subhead1-r.category-item")))

        button_subcategory = subcategories_elements[j].find_element(By.CSS_SELECTOR, ".category-item__link")
        driver.execute_script("arguments[0].scrollIntoView(true);", button_subcategory)
        driver.execute_script("arguments[0].click();", button_subcategory)

        time.sleep(2)

        temp_html = driver.page_source
        temp_soup = BeautifulSoup(temp_html, 'html.parser')
        print(category_name, subcategory_name)
        products = get_products_inf(temp_soup)
        print(len(products))
        for product in products:
            df.loc[len(df)] = [category_name, subcategory_name] + product
        
    time.sleep(2)
    
    
# Cerrar el navegador después de terminar
driver.quit()

Aceite, especias y salsas Aceite, vinagre y sal
35
Aceite, especias y salsas Mayonesa, ketchup y mostaza
50
Aceite, especias y salsas Otras salsas
21
Agua y refrescos Isotónico y energético
44
Agua y refrescos Refresco de cola
37
Agua y refrescos Refresco de naranja y de limón
38
Agua y refrescos Tónica y bitter
36
Agua y refrescos Refresco de té y sin gas
14
Aperitivos Frutos secos y fruta desecada
45
Aperitivos Patatas fritas y snacks
67
Arroz, legumbres y pasta Legumbres
17
Arroz, legumbres y pasta Pasta y fideos
21
Azúcar, caramelos y chocolate Chicles y caramelos
15
Azúcar, caramelos y chocolate Chocolate
31
Azúcar, caramelos y chocolate Golosinas
82
Azúcar, caramelos y chocolate Mermelada y miel
26
Azúcar, caramelos y chocolate Turrones
20
Bebé Biberón y chupete
43
Bebé Higiene y cuidado
9
Bebé Toallitas y pañales
16
Bodega Cerveza sin alcohol
76
Bodega Licores
20
Bodega Sidra y cava
48
Bodega Tinto de verano y sangría
12
Bodega Vino blanco
13
Bodega Vino lambrusco y espumoso
30


In [160]:
df.to_csv('alimentos_mercadona.csv', index=False)